In [1]:
import math
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import sentencepiece as spm
from datasets import load_dataset
import sacrebleu
from tqdm import trange, tqdm

In [2]:
# Hyperparameters
MAX_SEQ_LEN = 64
D_MODEL = 64          # Embedding dimension
HEADS = 4                  # Number of attention heads
N_LAYERS = 2           # Number of encoder/decoder layers
D_FF = 256             # Feed-forward dimension
DROPOUT = 0.1
BATCH_SIZE = 64        # Adjust if OOM
VOCAB_SIZE = 8000   # SentencePiece vocab size
SP_MODEL_PREFIX = 'spm'
PAD_IDX = 0
BEAM_WIDTH = 4
LENGTH_PENALTY = 0.6
SMOOTHING_EPS = 0.1
WARMUP = 4000

In [3]:
# Function to train SentencePiece on your Flores texts
def train_sentencepiece(texts, model_prefix=SP_MODEL_PREFIX, vocab_size=VOCAB_SIZE):
    
    input_file = f"{model_prefix}_input.txt"
    # write all sentences
    with open(input_file, 'w', encoding='utf-8') as f:
        for line in texts:
            f.write(line.strip() + "\n")
    # train BPE model with designated special IDs
    spm.SentencePieceTrainer.Train(
        f"--input={input_file} --model_prefix={model_prefix} "
        f"--vocab_size={vocab_size} --character_coverage=1.0 --model_type=bpe "
        "--pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3"
        "--minloglevel=2"
    )
    sp = spm.SentencePieceProcessor()
    sp.Load(f"{model_prefix}.model")
    # cleanup
    os.remove(input_file)
    return sp

In [4]:
# Encode with SP, adding BOS/EOS and padding
def encode_sp(text, sp, max_len=MAX_SEQ_LEN):
    ids = sp.EncodeAsIds(text)
    # BOS + tokens + EOS, trimmed
    seq = [sp.bos_id()] + ids[: max_len - 2] + [sp.eos_id()]
    # pad
    seq += [sp.pad_id()] * (max_len - len(seq))
    return seq

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=DROPOUT, max_len=MAX_SEQ_LEN):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x): return self.dropout(x + self.pe[:, :x.size(1)])

In [6]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout=DROPOUT): super().__init__(); self.dropout = nn.Dropout(dropout)
    def forward(self, q, k, v, mask=None):
        d_k = q.size(-1)
        scores = q @ k.transpose(-2, -1) / math.sqrt(d_k)
        if mask is not None: scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = F.softmax(scores, dim=-1); attn = self.dropout(attn)
        return attn @ v, attn

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=DROPOUT):
        super().__init__(); assert d_model % heads == 0
        self.d_k = d_model // heads; self.h = heads
        self.q_lin = nn.Linear(d_model, d_model)
        self.k_lin = nn.Linear(d_model, d_model)
        self.v_lin = nn.Linear(d_model, d_model)
        self.attn = ScaledDotProductAttention(dropout)
        self.out = nn.Linear(d_model, d_model)
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)
        q = self.q_lin(q).view(bs, -1, self.h, self.d_k).transpose(1, 2)
        k = self.k_lin(k).view(bs, -1, self.h, self.d_k).transpose(1, 2)
        v = self.v_lin(v).view(bs, -1, self.h, self.d_k).transpose(1, 2)
        # if mask is not None: mask = mask.unsqueeze(1)
        x, _ = self.attn(q, k, v, mask)
        x = x.transpose(1, 2).contiguous().view(bs, -1, self.h * self.d_k)
        return self.out(x)

In [8]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=DROPOUT):
        super().__init__(); self.lin1 = nn.Linear(d_model, d_ff)
        self.lin2 = nn.Linear(d_ff, d_model); self.drop = nn.Dropout(dropout)
    def forward(self, x): return self.lin2(self.drop(F.relu(self.lin1(x))))

In [9]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout=DROPOUT):
        super().__init__()
        self.self_attn = MultiHeadAttention(heads, d_model, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model); self.norm2 = nn.LayerNorm(d_model)
        self.drop1 = nn.Dropout(dropout); self.drop2 = nn.Dropout(dropout)
    def forward(self, x, mask):
        _x = self.norm1(x)
        x = x + self.drop1(self.self_attn(_x, _x, _x, mask))
        _x = self.norm2(x)
        return x + self.drop2(self.ff(_x))

In [10]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout=DROPOUT):
        super().__init__()
        self.self_attn = MultiHeadAttention(heads, d_model, dropout)
        self.cross_attn = MultiHeadAttention(heads, d_model, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model); self.norm2 = nn.LayerNorm(d_model); self.norm3 = nn.LayerNorm(d_model)
        self.drop1 = nn.Dropout(dropout); self.drop2 = nn.Dropout(dropout); self.drop3 = nn.Dropout(dropout)
    def forward(self, x, mem, tgt_mask, mem_mask):
        _x = self.norm1(x)
        x = x + self.drop1(self.self_attn(_x, _x, _x, tgt_mask))
        _x = self.norm2(x)
        x = x + self.drop2(self.cross_attn(_x, mem, mem, mem_mask))
        _x = self.norm3(x)
        return x + self.drop3(self.ff(_x))


In [11]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, d_ff, dropout=DROPOUT):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_enc = PositionalEncoding(d_model, dropout)
        self.layers = nn.ModuleList([EncoderLayer(d_model, heads, d_ff, dropout) for _ in range(N)])
        self.norm = nn.LayerNorm(d_model)
    def forward(self, src, mask):
        x = self.embed(src) * math.sqrt(self.embed.embedding_dim)
        x = self.pos_enc(x)
        for layer in self.layers: x = layer(x, mask)
        return self.norm(x)

In [12]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, d_ff, dropout=DROPOUT):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_enc = PositionalEncoding(d_model, dropout)
        self.layers = nn.ModuleList([DecoderLayer(d_model, heads, d_ff, dropout) for _ in range(N)])
        self.norm = nn.LayerNorm(d_model)
    def forward(self, tgt, mem, tgt_mask, mem_mask):
        x = self.embed(tgt) * math.sqrt(self.embed.embedding_dim)
        x = self.pos_enc(x)
        for layer in self.layers: x = layer(x, mem, tgt_mask, mem_mask)
        return self.norm(x)

In [13]:
class Transformer(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, d_model=D_MODEL, n_layers=N_LAYERS, heads=HEADS, d_ff=D_FF, dropout=DROPOUT):
        super().__init__()
        self.encoder = Encoder(vocab_size, d_model, n_layers, heads, d_ff, dropout)
        self.decoder = Decoder(vocab_size, d_model, n_layers, heads, d_ff, dropout)
        self.out = nn.Linear(d_model, vocab_size)
        # tie weights
        self.out.weight = self.decoder.embed.weight
    def forward(self, src, tgt, src_mask, tgt_mask, mem_mask):
        mem = self.encoder(src, src_mask)
        dec = self.decoder(tgt, mem, tgt_mask, mem_mask)
        return self.out(dec)

In [14]:
def create_masks(src, tgt, pad_idx=PAD_IDX):
    # src: (B, S), tgt: (B, T)
    # 1) Encoder padding mask: (B,1,1,S)
    src_mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)

    # 2) Decoder padding mask: (B,1,1,T)
    tgt_pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(2)

    # 3) Subsequent mask: (1,1,T,T)
    T = tgt.size(1)
    subsequent = torch.triu(torch.ones((T, T), device=tgt.device), diagonal=1).bool()
    subsequent_mask = subsequent.unsqueeze(0).unsqueeze(1)

    # 4) Combine: now (B,1,T,T)
    tgt_mask = tgt_pad_mask & ~subsequent_mask

    # 5) Cross-attention (same as encoder mask): (B,1,1,S)
    memory_mask = src_mask

    return src_mask, tgt_mask, memory_mask


In [15]:
class TranslationDataset(Dataset):
    def __init__(self, src, tgt): self.src, self.tgt = src, tgt
    def __len__(self): return len(self.src)
    def __getitem__(self, i): return self.src[i], self.tgt[i]

In [16]:
def beam_search(model, src, src_mask, sp, device, beam_width=BEAM_WIDTH, max_len=MAX_SEQ_LEN, alpha=LENGTH_PENALTY):
    model.eval()
    memory = model.encoder(src, src_mask)
    beams = [([sp.bos_id()], 0.0)]
    completed = []

    for _ in range(max_len):
        new_beams = []
        for tokens, score in beams:
            # If already ended, collect and skip expansion
            if tokens[-1] == sp.eos_id():
                completed.append((tokens, score))
                continue

            # Prepare decoder input and masks
            ys = torch.tensor([tokens], device=device)
            tgt_pad = (ys != sp.pad_id()).unsqueeze(1).unsqueeze(2)
            T = ys.size(1)
            sub = torch.triu(torch.ones((T, T), device=device), 1).bool()
            sub = sub.unsqueeze(0).unsqueeze(1)
            tgt_mask = tgt_pad & ~sub

            # One-step decoding
            dec = model.decoder(ys, memory, tgt_mask, src_mask)
            logits = model.out(dec[:, -1, :])
            log_probs = F.log_softmax(logits, dim=-1).squeeze(0)
            topk = torch.topk(log_probs, beam_width)

            # Expand each beam
            for logp, idx in zip(topk.values.tolist(), topk.indices.tolist()):
                # Skip immediate repetitions to avoid loops
                if idx == tokens[-1]:
                    continue
                new_beams.append((tokens + [idx], score + logp))

        # Prune to keep top beams by length-penalized score
        beams = sorted(new_beams, key=lambda x: x[1] / (len(x[0]) ** alpha), reverse=True)[:beam_width]
        if not beams:
            break

    # Add any completed beams
    beams.extend(completed)
    # Select best overall (with length normalization)
    best = max(beams, key=lambda x: x[1] / (len(x[0]) ** alpha))

    # Strip BOS/EOS and decode
    return sp.DecodeIds(best[0][1:-1])  # remove bos and eos

In [17]:
from torch.optim.lr_scheduler import LambdaLR
# Label smoothing loss
def get_label_smoothing_loss(vocab_size, ignore_index, eps=SMOOTHING_EPS):
    class LabelSmoothingLoss(nn.Module):
        def __init__(self):
            super().__init__()
            self.conf = 1.0 - eps
            self.smooth = eps
            self.vocab_size = vocab_size
            self.ignore = ignore_index
        def forward(self, pred, target):
            # pred: (N, V), target: (N,)
            with torch.no_grad():
                true_dist = torch.full_like(pred, self.smooth / (self.vocab_size - 1))
                true_dist.scatter_(1, target.unsqueeze(1), self.conf)
                mask = target == self.ignore
                true_dist[mask] = 0
            return torch.mean(torch.sum(-true_dist * F.log_softmax(pred, dim=1), dim=1))
    return LabelSmoothingLoss()

# Noam schedule
def noam_schedule(step, d_model=D_MODEL, warmup=WARMUP):
    step = step + 1
    return (d_model ** -0.5) * min(step ** -0.5, step * warmup ** -1.5)


In [18]:
def evaluate_bleu(model, data_iter, sp, device):
    """
    Evaluate corpus BLEU over a DataLoader or iterable of (src_ids, tgt_ids) pairs with progress bar.
    """
    refs, hyps = [], []
    # Iterate through batches or dataset
    for batch in tqdm(data_iter, desc="Evaluating BLEU", unit="batch"):
        # Unpack src and tgt
        src_batch, tgt_batch = batch
        # src_batch: tensor (B, L), tgt_batch: tensor (B, L)
        with torch.no_grad():
            for src_ids, tgt_ids in zip(src_batch, tgt_batch):
                src = src_ids.unsqueeze(0).to(device)
                src_mask = (src != sp.pad_id()).unsqueeze(1).unsqueeze(2)
                hyp = beam_search(model, src, src_mask, sp, device)
                # reference text
                # print(tgt_ids)
                tgt_seq = [i for i in tgt_ids.tolist() if i not in (sp.pad_id(), sp.bos_id(), sp.eos_id())]
                ref = sp.DecodeIds(tgt_seq)
                hyp = hyp.replace(" .", ".").replace(" ,", ",").replace(" !", "!").replace(" ?", "?")
                ref = ref.replace(" .", ".").replace(" ,", ",").replace(" !", "!").replace(" ?", "?")
                hyps.append(hyp)
                refs.append(ref)
    for i in range (5):
        print(f"Prediction: {hyps[i]}")
        print(f"Ground Truth: {refs[i]}")
    bleu = sacrebleu.corpus_bleu(hyps, [refs])
    print(f"BLEU score: {bleu.score:.2f}")
    return bleu.score

In [19]:
def prepare_data(batch_size=BATCH_SIZE, vocab_size=VOCAB_SIZE):
    """
    Load HF dataset, train SentencePiece, and return DataLoader + SP processor + vocab info.
    """
    data = load_dataset("SEACrowd/ted_en_id", trust_remote_code=True)
    data_train = data['train']
    data_test = data['test']
    # collect all sentences for SP training (both directions)
    all_texts = [ex['text'] for ex in data_train] + [ex['label'] for ex in data_train]
    sp = train_sentencepiece(all_texts, vocab_size=vocab_size)
    vocab_size = sp.GetPieceSize()
    pad_idx = sp.pad_id()
    # encode dataset
    src_list = [torch.tensor(encode_sp(ex['text'], sp), dtype=torch.long) for ex in data_train]
    tgt_list = [torch.tensor(encode_sp(ex['label'], sp), dtype=torch.long) for ex in data_train]
    train_loader = DataLoader(TranslationDataset(src_list, tgt_list), batch_size=batch_size, shuffle=True)
    src_list = [torch.tensor(encode_sp(ex['text'], sp), dtype=torch.long) for ex in data_test]
    tgt_list = [torch.tensor(encode_sp(ex['label'], sp), dtype=torch.long) for ex in data_test]
    test_loader = DataLoader(TranslationDataset(src_list, tgt_list), batch_size=batch_size, shuffle=False)
    return train_loader, test_loader, sp, vocab_size, pad_idx

In [20]:
def train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100):
    """
    Train the Transformer model with label smoothing and Noam schedule.
    """
    # prepare criterion, optimizer, scheduler
    criterion = get_label_smoothing_loss(vocab_size, pad_idx, eps=SMOOTHING_EPS)
    optimizer = torch.optim.Adam(model.parameters(), lr=1.0, betas=(0.9, 0.98), eps=1e-9)
    scheduler = LambdaLR(optimizer, lr_lambda=lambda step: noam_schedule(step, warmup=WARMUP))

    pbar = trange(1, epochs + 1, desc="Training", unit="epoch")
    for epoch in pbar:
        model.train()
        total_loss = 0.0
        for step, (src_batch, tgt_batch) in enumerate(train_loader, start=1):
            src_batch, tgt_batch = src_batch.to(device), tgt_batch.to(device)
            tgt_input = tgt_batch[:, :-1]
            s_mask, t_mask, m_mask = create_masks(src_batch, tgt_input, pad_idx)
            logits = model(src_batch, tgt_input, s_mask, t_mask, m_mask)
            loss = criterion(logits.view(-1, vocab_size), tgt_batch[:, 1:].reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(loss=f"{avg_loss:.4f}", lr=f"{lr:.6f}")

    return model

# EN -> ID

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer().to(device)

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit

In [22]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      1,265,728


In [23]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [34:55<00:00, 20.95s/epoch, loss=1.5462, lr=0.000338]


In [24]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [19:23<00:00, 23.27s/batch]


Prediction: Pada akhir tahun ini, ada di planet ini akan ada orang-orang di planet sosial yang menggunakan jaringan sosial.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Salah satu hal yang umum di semua itu adalah mereka akan mati.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Mungkin itu mungkin menjadi moralitas, saya berpikir bahwa hal itu sangat penting.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama saya pikirkan tentang blog ini adalah sebuah blog ini oleh D. Rill, yang telah meninggal pada tahun Mill, dan ilmuwan yang meninggal.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi, yang meninggal karena kanker.
Prediction: Dan

13.820058819427818

## Vocabulary Size

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data(vocab_size=4000)
model = Transformer(vocab_size=4000).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=4000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [23]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      749,728


In [24]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|██████████| 100/100 [35:22<00:00, 21.23s/epoch, loss=1.6680, lr=0.000338]


In [39]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|██████████| 50/50 [20:18<00:00, 24.36s/batch]


Prediction: Di akhir tahun ini , ada beberapa milyar orang yang akan duduk di planet ini .
Ground Truth: Pada akhir tahun ini , akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif .
Prediction: Satu hal yang sama dengan mereka adalah bahwa mereka akan mati .
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal .
Prediction: Siapa yang mungkin adalah sebuah pagi , saya pikir , saya berpikir bahwa saya pikir itu benar-benar mengeksplorasi .
Ground Truth: Walau itu pemikiran yang agak menakutkan , saya pikir ada implikasi yang mendalam yang perlu ditelusuri .
Prediction: Apa yang pertama saya pikirkan tentang saya tentang ini adalah seorang profesor ini adalah seorang wanita yang telah meninggal oleh Kh , dan ilmu pengetahuan yang meninggal .
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller , seorang jurnalis ilmiah dan teknologi , yang mening

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


BLEU score (beam=4): 12.65


12.64693430106625

## Model Dimension

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer(d_model=32).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [42]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      612,800


In [43]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|██████████| 100/100 [33:52<00:00, 20.33s/epoch, loss=1.7332, lr=0.000338]


In [46]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|██████████| 50/50 [18:44<00:00, 22.49s/batch]


Prediction: Dalam akhir tahun ini, ada di sini, ada di planet ini di planet ini di planet sosial.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Hal ini adalah satu hal yang harus mereka lakukan di sana.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Apakah hal ini mungkin berpikir bahwa saya pikir, saya pikir bahwa saya pikir bahwa hal yang sangat penting.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama saya berpikir tentang apa yang pertama ini adalah sebuah film yang telah dilakukan oleh para ilmuwan yang meninggal, yang meninggal oleh para ahli fisika, dan teknologi yang meninggal.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi

9.694397018993156

## Number of Layers

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer(n_layers=1).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [45]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      1,148,992


In [46]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [21:20<00:00, 12.81s/epoch, loss=1.6207, lr=0.000338]


In [47]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [11:56<00:00, 14.33s/batch]


Prediction: Pada akhir tahun ini, ada satu miliar orang yang akan digunakan di planet ini.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Salah satu hal yang umum adalah mereka akan meninggal di mana mereka meninggal.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Apa yang mungkin saya berpikir bahwa saya berpikir bahwa saya berpikir bahwa beberapa hal yang sangat penting.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama saya berpikir tentang blog ini adalah sebuah blog yang telah dilakukan oleh Dil, D. D. bernama D. dan teknologi yang meninggal.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi, yang meninggal karena kanker.
Predictio

10.394259656467081

## Number of Heads

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer(heads=2).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [49]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      1,265,728


In [50]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [33:46<00:00, 20.26s/epoch, loss=1.5423, lr=0.000338]


In [51]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [21:17<00:00, 25.56s/batch]


Prediction: Pada akhir tahun ini, ada satu miliar orang di planet ini yang menggunakan jaringan sosial yang menggunakan jaringan sosial.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Salah satu hal yang kita miliki di mana mereka akan mati.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Apapun yang mungkin menjadi sangat penting, saya pikir saya berpikir bahwa hal ini sangat besar.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama yang pertama saya pikirkan tentang blog ini adalah blog di Dill, Dill, seorang jurnalis, yang seorang jurnalis yang meninggal.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi, yang meninggal karena kanker.


14.36012738155998

# Softpick

In [21]:
def softpick(x: torch.Tensor, *, dim: int = -1, eps: float = 1e-6) -> torch.Tensor:
    """
    SoftPick:  ReLU(exp(x) - 1) / Σ|exp(x) - 1|
    • identical interface to F.softmax
    • numerically stable on large |x| by subtracting max(x)
    • differentiable everywhere except at the ReLU knee (same as ReLU)

    Args
    ----
    x   : (..., n) – raw attention scores
    dim : axis that should sum to 1
    eps : keeps gradients finite when the entire row is exactly zero
    """
    x_m = torch.max(x, dim=dim, keepdim=True).values
    x_m_e_m = torch.exp(-x_m)
    x_e_1 = torch.exp(x - x_m) - x_m_e_m
    r_x_e_1 = F.relu(x_e_1)
    a_x_e_1 = torch.where(x.isfinite(), torch.abs(x_e_1), 0)
    return r_x_e_1 / (torch.sum(a_x_e_1, dim=dim, keepdim=True) + eps)

In [22]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout=DROPOUT): super().__init__(); self.dropout = nn.Dropout(dropout)
    def forward(self, q, k, v, mask=None):
        d_k = q.size(-1)
        scores = q @ k.transpose(-2, -1) / math.sqrt(d_k)
        if mask is not None: scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = softpick(scores, dim=-1); attn = self.dropout(attn)
        return attn @ v, attn

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer().to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [24]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      1,265,728


In [25]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [38:54<00:00, 23.34s/epoch, loss=1.5406, lr=0.000338]


In [26]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [27:39<00:00, 33.19s/batch]


Prediction: Pada akhir tahun ini, ada beberapa miliar orang di planet ini di planet sosial yang menggunakan jaringan sosial.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Salah satu hal yang sama dari mereka memiliki mereka akan mati.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Apakah hal itu mungkin menjadi sebuah moralitas, saya pikir bahwa saya benar-benar memiliki dampak yang sangat penting.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama kali saya berpikir tentang blog ini adalah sebuah blog dari D. D. D. D. dan seorang jurnalis yang meninggal oleh teknologi yang meninggal.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi, y

13.974490198992573

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data(vocab_size=4000)
model = Transformer(vocab_size=4000).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=4000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [28]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      749,728


In [29]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [40:51<00:00, 24.51s/epoch, loss=1.6524, lr=0.000338]


In [30]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [28:09<00:00, 33.79s/batch]


Prediction: Pada akhir tahun ini, ada sekitar 1 miliar orang di planet ini di planet ini.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Yang salah satu hal yang ada di dalam mereka adalah bahwa mereka akan meninggal.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Siapa yang mungkin ada seorang profesor, saya berpikir bahwa saya pikir ini sangat buruk.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama saya pikirkan adalah tentang otomatis ini adalah per tahun ini oleh D. D. D. D. Mart, seorang ilmuwan yang telah meninggal.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi, yang meninggal karena kanker.
Prediction: Dan apa yang dilakukan

12.915632463105004

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer(d_model=32).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [32]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      612,800


In [33]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [39:42<00:00, 23.82s/epoch, loss=1.7400, lr=0.000338]


In [34]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [26:01<00:00, 31.24s/batch]


Prediction: Pada tahun ini, ada beberapa miliar orang-orang yang akan berada di planet ini di planet ini di planet ini.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Salah satu hal yang lain adalah bahwa mereka akan mati.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Apakah itu mungkin akan benar-benar benar-benar benar-benar berpikir, saya berpikir bahwa beberapa hal yang sangat besar.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama tentang saya tentang buku ini adalah sebuah buku yang baru yang telah saya, dengan teknologi yang telah bekerja dengan teknologi ini, dan pemerintah.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi, y

9.496860251142657

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer(n_layers=1).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [36]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      1,148,992


In [37]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [24:11<00:00, 14.51s/epoch, loss=1.6227, lr=0.000338]


In [38]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [15:15<00:00, 18.31s/batch]


Prediction: Pada tahun ini, ada satu miliar orang-orang di planet ini akan menggunakan jaringan sosial.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Yang satu hal yang sama yang akan meninggal.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Mungkin itu mungkin menjadi hal itu, saya berpikir bahwa saya berpikir bahwa saya berpikir bahwa hal yang sangat besar.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama saya pikirkan tentang blog ini adalah sebuah blog yang telah meninggal oleh seorang ilmuwan yang telah meninggal pada tahun yang telah meninggal oleh R. R. dan Mesir.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan teknologi, yang meningga

9.892148829116039

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader, sp, vocab_size, pad_idx = prepare_data()
model = Transformer(heads=2).to(device)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=spm_input.txt --model_prefix=spm --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3--minloglevel=2
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: spm_input.txt
  input_format: 
  model_prefix: spm
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_

In [40]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters:      {total_params:,}")

Total parameters:      1,265,728


In [41]:
trained_model = train_model(model, train_loader, sp, vocab_size, pad_idx, device, epochs=100)

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [40:34<00:00, 24.35s/epoch, loss=1.5546, lr=0.000338]


In [42]:
evaluate_bleu(trained_model, test_loader, sp, device)

Evaluating BLEU: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [25:03<00:00, 30.06s/batch]


Prediction: Pada akhir tahun ini, ada hampir 1 miliar orang akan duduk di planet ini di planet ini.
Ground Truth: Pada akhir tahun ini, akan ada hampir satu milyar orang di planet ini yang menggunakan situs jejaring sosial secara aktif.
Prediction: Yang salah satu hal yang ada di seluruh mereka akan mati.
Ground Truth: Satu kesamaan dari mereka semua adalah mereka akan meninggal.
Prediction: Apapun yang mungkin menjadi ide yang mungkin saya pikir, saya pikir bahwa hal ini sangat penting yang sangat penting.
Ground Truth: Walau itu pemikiran yang agak menakutkan, saya pikir ada implikasi yang mendalam yang perlu ditelusuri.
Prediction: Apa yang pertama saya pikirkan tentang blog ini adalah sebuah blog ini adalah sebuah blog ini adalah seorang jurnalis bernama KB, yang meninggal, yang meninggal oleh para jurnalis yang meninggal.
Ground Truth: Yang membuat saya mulai berpikir seperti ini adalah sebuah tulisan blog pada awal tahun ini dari Derek K. Miller, seorang jurnalis ilmiah dan tekno

13.516293466216013